In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.cross_validation import train_test_split
import matplotlib.pylab as plt
from sklearn import preprocessing
from sklearn.metrics import log_loss
from sklearn.metrics import make_scorer
from sklearn.preprocessing import StandardScaler
from matplotlib.colors import LogNorm
from sklearn.decomposition import PCA
from sklearn.metrics import roc_auc_score
from copy import deepcopy
import xgboost as xgb
%matplotlib inline  

In [2]:
train_df=pd.read_csv("../../train.csv")

In [3]:
train_df=train_df.fillna(-1)

In [4]:
print train_df.head()

   Id  Product_Info_1 Product_Info_2  Product_Info_3  Product_Info_4  \
0   2               1             D3              10        0.076923   
1   5               1             A1              26        0.076923   
2   6               1             E1              26        0.076923   
3   7               1             D4              10        0.487179   
4   8               1             D2              26        0.230769   

   Product_Info_5  Product_Info_6  Product_Info_7   Ins_Age        Ht  \
0               2               1               1  0.641791  0.581818   
1               2               3               1  0.059701  0.600000   
2               2               3               1  0.029851  0.745455   
3               2               3               1  0.164179  0.672727   
4               2               3               1  0.417910  0.654545   

     ...     Medical_Keyword_40  Medical_Keyword_41  Medical_Keyword_42  \
0    ...                      0                   0  

In [5]:
print train_df.describe()

                 Id  Product_Info_1  Product_Info_3  Product_Info_4  \
count  59381.000000    59381.000000    59381.000000    59381.000000   
mean   39507.211515        1.026355       24.415655        0.328952   
std    22815.883089        0.160191        5.072885        0.282562   
min        2.000000        1.000000        1.000000        0.000000   
25%    19780.000000        1.000000       26.000000        0.076923   
50%    39487.000000        1.000000       26.000000        0.230769   
75%    59211.000000        1.000000       26.000000        0.487179   
max    79146.000000        2.000000       38.000000        1.000000   

       Product_Info_5  Product_Info_6  Product_Info_7       Ins_Age  \
count    59381.000000    59381.000000    59381.000000  59381.000000   
mean         2.006955        2.673599        1.043583      0.405567   
std          0.083107        0.739103        0.291949      0.197190   
min          2.000000        1.000000        1.000000      0.000000   
25%  

In [14]:
features=train_df.drop(["Id","Response"],axis=1)
labels=train_df["Response"].astype("category")
labels=labels.cat.rename_categories(range(8))

In [15]:
print labels.unique()

[0 1 2 3 4 5 6 7]


### Only one categorical feature, use l

In [16]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()

In [17]:
features["Product_Info_2"]=le.fit_transform(features["Product_Info_2"])

In [18]:
print len(features)

59381


In [19]:
def xgboost_train(train,labels,num_class):
    params = {}
    params["objective"] = "multi:softprob"
    params["eta"] =  0.01
    params["subsample"] = 0.8
    params["colsample_bytree"] = 0.8
    params["num_class"]=num_class
#     params["scale_pos_weight"] = 1
    params["silent"] = 1
    params["max_depth"] = 12
    params["eval_metric"] = 'mlogloss'

    plst = list(params.items())

    offset = 10000

    num_rounds = 500

    #create a train and validation dmatrices 
    xgtrain = xgb.DMatrix(train[offset:,:], label=labels[offset:])
    xgval = xgb.DMatrix(train[:offset,:], label=labels[:offset])

    #train using early stopping and predict
    watchlist = [(xgtrain, 'train'),(xgval, 'val')]
    model1 = xgb.train(plst, xgtrain, num_rounds, watchlist, early_stopping_rounds=120,verbose_eval=True)


    #reverse train and labels and use different 5k for early stopping. 
    # this adds very little to the score but it is an option if you are concerned about using all the data. 
    train = train[::-1,:]
    # labels = np.log(labels[::-1])
    labels=labels[::-1]

    xgtrain = xgb.DMatrix(train[offset:,:], label=labels[offset:])
    xgval = xgb.DMatrix(train[:offset,:], label=labels[:offset])

    watchlist = [(xgtrain, 'train'),(xgval, 'val')]
    model2 = xgb.train(plst, xgtrain, num_rounds, watchlist, early_stopping_rounds=120,verbose_eval=True)
    
    return model1, model2

In [20]:
model1,model2 = xgboost_train(features.as_matrix(),labels.as_matrix(),num_class=len(labels.unique()))

Will train until val error hasn't decreased in 120 rounds.
[0]	train-mlogloss:2.064344	val-mlogloss:2.066763
[1]	train-mlogloss:2.049622	val-mlogloss:2.054656
[2]	train-mlogloss:2.036641	val-mlogloss:2.044050
[3]	train-mlogloss:2.022826	val-mlogloss:2.032485
[4]	train-mlogloss:2.008460	val-mlogloss:2.020583
[5]	train-mlogloss:1.995332	val-mlogloss:2.009737
[6]	train-mlogloss:1.981348	val-mlogloss:1.998270
[7]	train-mlogloss:1.967961	val-mlogloss:1.987317
[8]	train-mlogloss:1.954661	val-mlogloss:1.976287
[9]	train-mlogloss:1.941863	val-mlogloss:1.965850
[10]	train-mlogloss:1.929399	val-mlogloss:1.955621
[11]	train-mlogloss:1.917660	val-mlogloss:1.946016
[12]	train-mlogloss:1.905923	val-mlogloss:1.936666
[13]	train-mlogloss:1.894313	val-mlogloss:1.927308
[14]	train-mlogloss:1.883138	val-mlogloss:1.918228
[15]	train-mlogloss:1.871586	val-mlogloss:1.908738
[16]	train-mlogloss:1.860664	val-mlogloss:1.899783
[17]	train-mlogloss:1.849972	val-mlogloss:1.891059
[18]	train-mlogloss:1.839676	val-

In [32]:
test_df=pd.read_csv("../../test.csv")
test_df=test_df.fillna(-1)
features_sub=test_df.drop(["Id"],axis=1)
features_sub["Product_Info_2"]=le.transform(features_sub["Product_Info_2"])
xgtest=xgb.DMatrix(features_sub)

In [33]:
test_df.head()

,Id,Product_Info_1,Product_Info_2,Product_Info_3,Product_Info_4,Product_Info_5,Product_Info_6,Product_Info_7,Ins_Age,Ht,...,Medical_Keyword_39,Medical_Keyword_40,Medical_Keyword_41,Medical_Keyword_42,Medical_Keyword_43,Medical_Keyword_44,Medical_Keyword_45,Medical_Keyword_46,Medical_Keyword_47,Medical_Keyword_48
0,1,1,D3,26,0.487179,2,3,1,0.611940,0.781818,...,0,0,0,0,0,0,0,0,0,0
1,3,1,A2,26,0.076923,2,3,1,0.626866,0.727273,...,0,0,0,0,0,0,0,0,0,0
2,4,1,D3,26,0.144667,2,3,1,0.582090,0.709091,...,0,0,0,0,0,0,0,0,0,0
3,9,1,A1,26,0.151709,2,1,1,0.522388,0.654545,...,0,0,0,0,0,0,0,0,1,1
4,12,1,A1,26,0.076923,2,3,1,0.298507,0.672727,...,0,0,0,0,0,0,0,0,0,0


In [34]:
preds1 = model1.predict(xgtest,ntree_limit=model1.best_iteration)
preds2 = model2.predict(xgtest,ntree_limit=model2.best_iteration)

In [39]:
pred_df=pd.DataFrame(0.5*preds1+0.5*preds2,columns=range(1,9))

In [40]:
pred_df.head()

,1,2,3,4,5,6,7,8
0,0.211944,0.215980,0.011853,0.012067,0.039036,0.184171,0.295572,0.029376
1,0.084745,0.054607,0.013012,0.024527,0.036254,0.202218,0.113387,0.471249
2,0.079607,0.083397,0.012266,0.012724,0.046201,0.431723,0.213520,0.120563
3,0.092198,0.074641,0.010198,0.011050,0.071542,0.140646,0.214697,0.385027
4,0.042908,0.069235,0.011159,0.019391,0.026133,0.093412,0.162423,0.575339


In [46]:
def get_cat(x):
    retval=x.idxmax()
    return retval

In [47]:
maxprob=pred_df.apply(get_cat,axis=1)

In [48]:
maxprob

0     7
1     8
2     6
3     8
4     8
5     8
6     8
7     8
8     7
9     8
10    8
11    8
12    4
13    8
14    6
...
19750    5
19751    8
19752    2
19753    6
19754    8
19755    6
19756    2
19757    8
19758    6
19759    8
19760    8
19761    8
19762    1
19763    2
19764    4
Length: 19765, dtype: int64

In [49]:
sub_df=pd.concat([test_df['Id'],maxprob],axis=1)

In [51]:
sub_df.columns=["Id","Respone"]

In [52]:
sub_df.head()

,Id,Respone
0,1,7
1,3,8
2,4,6
3,9,8
4,12,8


In [53]:
sub_df.to_csv("xgboost_classification.csv",index=False)